# MIOST like SSH field 
Creating the boundary condition field for 4Dvar QG over Hawai'i

In [ ]:
import xarray as xr 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import scipy.fftpack as fp
from scipy.interpolate import RegularGridInterpolator, griddata

import sys
sys.path.append("/bettik/bellemva/src/")
from functions import open_mfdataset_w

from datetime import datetime

import glob

from joblib import Parallel
from joblib import delayed as jb_delayed

from pyinterp import fill, Axis, TemporalAxis, Grid3D, Grid2D

from math import *

In [ ]:
n_workers = 12

#### Parameters 

In [ ]:
lambda_cut = 300
time_cut = 20

## 1) Loading datasets  

In [ ]:
lon_min = 185; lon_max = 205
lat_min=15; lat_max = 35

In [ ]:
# ### LIST OF FILES TO OPEN ###
# list_files = glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201205*") + \
# glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201206*") + \
# glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201207*") + \
# glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201208*") + \
# glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201209*") + \
# glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201210*") 
# list_files.sort()

# ### OPENING FILES ###
# ds = open_mfdataset_w(list_files,drop_variables=["mask","ssh_cor","ssh_dedac","ssh_hf","ssh_bar","ssh_igw"])#,chunks={'time': 24,'longitude':6,'latitude':6})

# ds = ds.sel(longitude = slice(lon_min,lon_max),latitude = slice(lat_min,lat_max),drop=True)

# # time_window = 24
# lat_lon_window = 6

# # Apply rolling mean to reduce high-frequency variations
# ds_filtered = (
#     ds.rolling(longitude=lat_lon_window, center=True).mean()
#     .rolling(latitude=lat_lon_window, center=True).mean()
# )

# ds_subsampled = ds_filtered.isel(time=slice(None, None, 24),
#                                  longitude=slice(None, None, 6),
#                                  latitude=slice(None, None, 6))

# ssh_bm = ds_subsampled.ssh_bm.load()

In [ ]:
### LIST OF FILES TO OPEN ###
list_files = glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201205*") + \
glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201206*") + \
glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201207*") + \
glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201208*") + \
glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201209*") + \
glob.glob("/bettik/bellemva/MITgcm/MITgcm_filtered_final/MITgcm_filt_201210*") 
list_files.sort()

### OPENING FILES ###
ds = open_mfdataset_w(list_files,drop_variables=["mask","ssh_cor","ssh_dedac","ssh_hf","ssh_bar","ssh_igw"])#,chunks={'time': 24,'longitude':6,'latitude':6})

### SELECTING REGION ### 
ds = ds.sel(longitude = slice(lon_min,lon_max),latitude = slice(lat_min,lat_max))

### SUBSAMPLING THE SSH ###
ds = ds.isel(time=slice(None, None, 24))
# ds = ds.rolling(time=24, center=True).mean()
# ds = ds.chunk({'longitude':6,'latitude':6})

# ssh_cor = ds.ssh_cor[::24,:,:].load()
ds = ds.coarsen(longitude=6,latitude=6,boundary="trim").mean()
ssh_bm = ds.ssh_bm.load()
# ssh_cor = ds.ssh_cor[::24,::6,::6].load()

## 2) Call to functions 

In [ ]:
%run ./functions.ipynb

## 3) Filtering in time

In [ ]:
array_time = ssh_bm.time.values
nt = array_time.size

In [ ]:
# PARAMETERS # 
wint = np.ones(3*nt)
gaspari = gaspari_cohn(np.arange(0,2*nt,1),nt,nt)
wint[:nt]=gaspari[:nt]
wint[2*nt:]=gaspari[nt:]

dt = 24*3600 # a day in seconds

w = fp.fftfreq(3*nt,dt)# seconds^-1
nw = w.size

w_cut = 1/(20*time_cut*3600)
H = (np.abs(w)<w_cut)
w_filtered = H*w

In [ ]:
idx_ocean = np.where(np.invert(np.isnan(ssh_bm[0,:,:].values))) # indexes of ocean pixels 

In [ ]:
# PROCESSING # 
ssh_filtered_time = np.array(Parallel(n_jobs=n_workers,backend='multiprocessing')(jb_delayed(apply_low_pass_time)(ssh_bm[:,i,j],wint,H) for i,j in zip(idx_ocean[0],idx_ocean[1])))

In [ ]:
# ARRAY TO STORE THE RESULTS # 
n_time = ssh_bm.shape[0]
n_latitude = ssh_bm.shape[1]
n_longitude = ssh_bm.shape[2]

array_ssh_filtered_time = np.nan*np.ones((n_time,n_latitude,n_longitude),dtype="float64")
array_ssh_filtered_time[:,idx_ocean[0],idx_ocean[1]]=ssh_filtered_time.T

In [ ]:
ds_ssh_filtered_time = ssh_bm.copy(deep=True,data=array_ssh_filtered_time)

## 3) Filtering in space

In [ ]:
res = apply_low_pass_space(ds_ssh_filtered_time)

In [ ]:
ds_ssh_filtered_time_space = ssh_bm.copy(deep=True,data=res)

In [ ]:
ds_ssh_filtered_time_space = ds_ssh_filtered_time_space.rename("ssh")

In [ ]:
for (i,date) in enumerate(ds_ssh_filtered_time_space.time.values):
    _ds_filtered = ds_ssh_filtered_time_space.isel(time=i)
    _ds_filtered = _ds_filtered.expand_dims('time')
    _date_str = date.astype("datetime64[h]").astype("str")
    # _ds_filtered.to_netcdf(f"/bettik/bellemva/miost_hawaii/miost_like_ssh/miost_like_{_date_str}.nc")
    _ds_filtered.to_netcdf(f"/bettik/bellemva/miost_hawaii/miost_like_ssh_coarse/miost_like_{_date_str}.nc")
    print(_date_str)


## 3) Comparison with MIOST real field 

In [ ]:
ds_miost = xr.open_mfdataset("/bettik/bellemva/miost_hawaii/miost_run/*.nc").load()

In [ ]:
ds_miost_like = xr.open_mfdataset("/bettik/bellemva/miost_hawaii/miost_like_ssh/*.nc").load()

In [ ]:
plt.hist(ds_miost.adt.values.flatten(),alpha=0.6,label="miost_run",bins=100)
plt.hist(ds_miost_like.ssh.values.flatten(),alpha=0.6,label="miost_like",bins=100)
plt.legend()

In [ ]:
import matplotlib
matplotlib.use("Agg")


for i in range (180):
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))

    # Use the pcolormesh method on ax[0] and ax[1]
    p0 = ax[0].pcolormesh(ds_miost.longitude, ds_miost.latitude, ds_miost.adt[i, :, :],vmin=0.9,vmax=1.4)
    p1 = ax[1].pcolormesh(ds_miost_like.longitude, ds_miost_like.latitude, ds_miost_like.ssh[i, :, :],vmin=0.6,vmax=1.1)

    #set aspect equal 
    ax[0].set_aspect("equal")
    ax[1].set_aspect("equal")

    #set title 
    date_miost_str = ds_miost.time[i].values.astype("datetime64[D]").astype("str")
    date_miost_like_str = ds_miost_like.time[i].values.astype("datetime64[D]").astype("str")

    ax[0].set_title(f"MIOST run : {date_miost_str}")
    ax[1].set_title(f"MIOST like run : {date_miost_like_str}")

    # Optionally add colorbars to each plot
    fig.colorbar(p0, ax=ax[0], orientation='vertical')
    fig.colorbar(p1, ax=ax[1], orientation='vertical')

    plt.savefig(f"./frames/{date_miost_like_str}.png")
    # plt.show()
    
